# 天池o2o优惠券使用预测

## 加载相关库

In [1]:
# Import libraries necessary for this project
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

## 导入数据

In [2]:
df_offline = pd.read_csv('./data/ccf_offline_stage1_train.csv', keep_default_na=False)
df_online = pd.read_csv('./data/ccf_online_stage1_train.csv', keep_default_na=False)
df_test = pd.read_csv('./data/ccf_offline_stage1_test_revised.csv', keep_default_na=False)

In [3]:
df_offline.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [4]:
df_offline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
User_id          int64
Merchant_id      int64
Coupon_id        object
Discount_rate    object
Distance         object
Date_received    object
Date             object
dtypes: int64(2), object(5)
memory usage: 93.7+ MB


## 简单统计

简单统计一下用户使用优惠券的情况

In [5]:
receive_and_buy = df_offline[(df_offline['Date_received'] != 'null') & (df_offline['Date'] != 'null')]
receive_not_buy = df_offline[(df_offline['Date_received'] != 'null') & (df_offline['Date'] == 'null')]
not_receive_and_buy = df_offline[(df_offline['Date_received'] == 'null') & (df_offline['Date'] != 'null')]
not_receive_not_buy = df_offline[(df_offline['Date_received'] == 'null') & (df_offline['Date'] == 'null')]

In [6]:
print('有优惠券，购买商品：{}'.format(receive_and_buy.shape[0]))
print('有优惠券，未购商品：{}'.format(receive_not_buy.shape[0]))
print('无优惠券，购买商品：{}'.format(not_receive_and_buy.shape[0]))
print('无优惠券，未购商品：{}'.format(not_receive_not_buy.shape[0]))

有优惠券，购买商品：75382
有优惠券，未购商品：977900
无优惠券，购买商品：701602
无优惠券，未购商品：0


可见，很多人（701602）购买商品却没有使用优惠券，也有很多人（977900）有优惠券但却没有使用，真正使用优惠券购买商品的人（75382）很少！所以，这个比赛的意义就是把优惠券送给真正可能会购买商品的人。

## 特征提取

In [7]:
print(df_offline.columns)

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date'],
      dtype='object')


### 打折率 Discount_rate

In [8]:
df_offline['Discount_rate'].unique()

array(['null', '150:20', '20:1', '200:20', '30:5', '50:10', '10:5',
       '100:10', '200:30', '20:5', '30:10', '50:5', '150:10', '100:30',
       '200:50', '100:50', '300:30', '50:20', '0.9', '10:1', '30:1',
       '0.95', '100:5', '5:1', '100:20', '0.8', '50:1', '200:10',
       '300:20', '100:1', '150:30', '300:50', '20:10', '0.85', '0.6',
       '150:50', '0.75', '0.5', '200:5', '0.7', '30:20', '300:10', '0.2',
       '50:30', '200:100', '150:5'], dtype=object)

打折率分为 3 种情况：

- 'null' 表示没有打折: -1
- [0,1] 表示折扣率: 0
- x:y 表示满x减y: 1

处理方式：

- 打折类型：getDiscountType()
- 折扣率：convertRate()
- 满多少：getDiscountTarget()
- 减多少：getDiscountReduce()

**Build functions to deal with the Discount_rate**

In [9]:
def getDiscountType(row):
    if row == 'null':
        return -1
    elif ':' in row:
        return 1
    else:
        return 0

In [10]:
def convertRate(row):
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1 - float(rows[1])/float(rows[0])
    else:
        return float(row)

In [11]:
def getDiscountTarget(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

In [12]:
def getDiscountReduce(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

In [13]:
def processData(df):
    df['Discount_type'] = df['Discount_rate'].apply(getDiscountType)
    df['Convert_rate'] = df['Discount_rate'].apply(convertRate)
    df['Discount_target'] = df['Discount_rate'].apply(getDiscountTarget)
    df['Discount_reduce'] = df['Discount_rate'].apply(getDiscountReduce)
    
    return df

In [14]:
df_offline = processData(df_offline)
df_test = processData(df_test)

In [15]:
df_offline = df_offline.drop(['Discount_rate'], axis=1)
df_test = df_test.drop(['Discount_rate'], axis=1)

In [16]:
df_offline.sample(10)

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Discount_type,Convert_rate,Discount_target,Discount_reduce
432973,4575554,7418,6998,null,20160523,null,1,0.900000,10,1
96041,1877504,6485,10323,0,20160327,null,1,0.750000,20,5
572669,2917710,7717,null,0,null,20160606,-1,1.000000,0,0
904758,2724121,7555,null,0,null,20160306,-1,1.000000,0,0
1267904,5405704,6650,12910,0,20160520,20160520,1,0.966667,30,1
540347,377955,55,6706,9,20160516,null,1,0.750000,20,5
1048323,7048779,2099,12034,0,20160128,null,1,0.900000,100,10
15915,880863,5341,11539,0,20160205,20160207,1,0.833333,30,5
43460,3543464,7884,7484,0,20160121,null,1,0.950000,20,1
190285,3584666,2099,null,0,null,20160525,-1,1.000000,0,0


### 距离 Distance

In [17]:
df_offline['Distance'].unique()

array(['0', '1', 'null', '2', '10', '4', '7', '9', '3', '5', '6', '8'],
      dtype=object)

- 将距离数据类型由str转换为int
- 将null转换为-1

In [18]:
df_offline['Distance'] = df_offline['Distance'].replace('null', -1).astype(int) 
df_test['Distance'] = df_test['Distance'].replace('null', -1).astype(int)

In [19]:
df_offline.sample(10)

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Discount_type,Convert_rate,Discount_target,Discount_reduce
454214,6742490,2277,1495,1,20160518,null,1,0.666667,30,10
237020,1230452,1403,11941,8,20160615,null,1,0.950000,20,1
105059,2713100,2934,null,1,null,20160228,-1,1.000000,0,0
718815,4933760,2225,null,0,null,20160418,-1,1.000000,0,0
808684,5650856,3621,2705,0,20160520,null,1,0.750000,20,5
1685593,5516663,4663,818,5,20160124,null,1,0.866667,150,20
600264,2384766,7354,null,1,null,20160405,-1,1.000000,0,0
506859,7040239,1433,null,0,null,20160530,-1,1.000000,0,0
122196,1312532,6963,null,0,null,20160116,-1,1.000000,0,0
750489,1682746,7555,17,0,20160125,null,1,0.833333,30,5


### 领券日期 Date_received

In [20]:
date_received = df_offline['Date_received'].unique()
date_received = sorted(date_received[date_received != 'null'])

In [21]:
date_buy = df_offline['Date'].unique()
date_buy = sorted(date_buy[date_buy != 'null'])

In [22]:
print('优惠卷收到日期从 {} 到 {}'.format(date_received[0], date_received[-1]))
print('消费日期从 {} 到 {}'.format(date_buy[0], date_buy[-1]))

优惠卷收到日期从 20160101 到 20160615
消费日期从 20160101 到 20160630


关于领劵日期的特征：
- weekday : {null, 1, 2, 3, 4, 5, 6, 7}
- weekday_type : {1, 0}（周六和周日为1，其他为0）
- weekday_1 : {1, 0, 0, 0, 0, 0, 0}
- weekday_2 : {0, 1, 0, 0, 0, 0, 0}
- weekday_3 : {0, 0, 1, 0, 0, 0, 0}
- weekday_4 : {0, 0, 0, 1, 0, 0, 0}
- weekday_5 : {0, 0, 0, 0, 1, 0, 0}
- weekday_6 : {0, 0, 0, 0, 0, 1, 0}
- weekday_7 : {0, 0, 0, 0, 0, 0, 1}

In [23]:
from datetime import date

In [24]:
def getWeekday(row):
    if row == 'null':
        return row
    else:
        return date(int(row[:4]), int(row[4:6]), int(row[6:])).weekday() + 1

In [25]:
df_offline['weekday'] = df_offline['Date_received'].astype(str).apply(getWeekday)
df_test['weekday'] = df_test['Date_received'].astype(str).apply(getWeekday)

In [26]:
df_offline['weekday_type'] = df_offline['weekday'].apply(lambda x: 1 if x in [6,7] else 0)
df_test['weekday_type'] = df_test['weekday'].apply(lambda x: 1 if x in [6,7] else 0)

In [27]:
# Change weekday to one_hot encoding
col_names = ['weekday_' + str(i) for i in range(1,8)]

tmp_df = pd.get_dummies(df_offline['weekday'].replace('null', np.nan))
tmp_df.columns = col_names
df_offline[col_names] = tmp_df

tmp_df = pd.get_dummies(df_test['weekday'].replace('null', np.nan))
tmp_df.columns = col_names
df_test[col_names] = tmp_df

In [28]:
df_offline = df_offline.drop(['weekday'], axis=1)
df_test = df_test.drop(['weekday'], axis=1)

In [29]:
df_offline.sample(10)

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Discount_type,Convert_rate,Discount_target,Discount_reduce,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
281933,5923829,760,null,0,null,20160503,-1,1.000000,0,0,0,0,0,0,0,0,0,0
1487323,2750729,7418,null,10,null,20160327,-1,1.000000,0,0,0,0,0,0,0,0,0,0
488169,5566399,7884,11797,0,20160517,null,1,0.950000,20,1,0,0,1,0,0,0,0,0
1407279,2138344,1433,9476,10,20160125,null,1,0.750000,20,5,0,1,0,0,0,0,0,0
1671129,5820563,6434,10050,0,20160125,null,1,0.900000,50,5,0,1,0,0,0,0,0,0
325858,7144022,2622,null,0,null,20160327,-1,1.000000,0,0,0,0,0,0,0,0,0,0
1207071,2784160,1982,null,0,null,20160203,-1,1.000000,0,0,0,0,0,0,0,0,0,0
1340237,1563198,1569,5054,-1,20160422,null,1,0.850000,200,30,0,0,0,0,0,1,0,0
125024,1943063,7717,null,0,null,20160203,-1,1.000000,0,0,0,0,0,0,0,0,0,0
600384,2538366,8181,4872,0,20160407,null,1,0.833333,30,5,0,0,0,0,1,0,0,0


所有特征：
- Convert_rate
- Discount_type
- Discount_target
- Discount_reduce
- Distance
- weekday_type
- weekday_1
- weekday_2
- weekday_3
- weekday_4
- weekday_5
- weekday_6
- weekday_7

## 标签标注

三种情况：

- Date_received == 'null'：表示没有领到优惠券，无需考虑，y = -1

- (Date_received != 'null') & (Date != 'null') & (Date - Date_received <= 15)：表示领取优惠券且在15天内使用，即正样本，y = 1

- (Date_received != 'null') & ((Date == 'null') | (Date - Date_received > 15))：表示领取优惠券未在在15天内使用，即负样本，y = 0

定义标签备注函数：

In [30]:
def label(row):
    if row['Date_received'] == 'null':
        return int(-1)
    if row['Date'] != 'null':
        td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return int(1)
    return int(0)

In [31]:
df_offline['label'] = df_offline.apply(label, axis=1)

In [32]:
print(df_offline['label'].value_counts())

 0    988887
-1    701602
 1     64395
Name: label, dtype: int64


In [33]:
df_offline.sample(10)

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Discount_type,Convert_rate,Discount_target,Discount_reduce,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label
513585,7236955,3447,6922,0,20160529,null,1,0.80,50,10,1,0,0,0,0,0,0,1,0
1508554,3830609,1569,5054,5,20160424,null,1,0.85,200,30,1,0,0,0,0,0,0,1,0
1647874,4840211,3575,null,0,null,20160415,-1,1.00,0,0,0,0,0,0,0,0,0,0,-1
1087009,5626035,6937,null,1,null,20160516,-1,1.00,0,0,0,0,0,0,0,0,0,0,-1
291595,4358297,6021,9588,0,20160304,null,1,0.95,20,1,0,0,0,0,0,1,0,0,0
645741,1448932,1715,null,3,null,20160326,-1,1.00,0,0,0,0,0,0,0,0,0,0,-1
1410733,3357424,3284,8643,0,20160323,20160326,1,0.75,20,5,0,0,0,1,0,0,0,0,1
1689512,6320567,760,null,0,null,20160528,-1,1.00,0,0,0,0,0,0,0,0,0,0,-1
722280,855697,3033,null,1,null,20160531,-1,1.00,0,0,0,0,0,0,0,0,0,0,-1
1254535,1693176,2709,12207,5,20160204,null,1,0.90,200,20,0,0,0,0,1,0,0,0,0


## 建立线性模型 SGDClassifier

- 使用上面提取的14个特征。

- 训练集：20160101-20160515；验证集：20160516-20160615。

- 用线性模型 SGDClassifier

### 划分训练集/验证集

In [34]:
df = df_offline[df_offline['label'] != -1].copy()
train = df[(df['Date_received'] < '20160516')].copy()
test = df[(df['Date_received'] >= '20160516') & (df['Date_received'] <= '20160615')].copy()

In [35]:
train['label'].value_counts()

0    759172
1     41524
Name: label, dtype: int64

In [36]:
test['label'].value_counts()

0    229715
1     22871
Name: label, dtype: int64

### 特征数量

In [37]:
factors = ['Convert_rate', 'Discount_type', 'Discount_target', 'Discount_reduce', 'Distance', 'weekday_type',
          'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']
print('共有{}个特征'.format(len(factors)))

共有13个特征


### 归一化

In [38]:
from sklearn.preprocessing import MinMaxScaler

In [39]:
scaler = MinMaxScaler()

train[factors] = scaler.fit_transform(train[factors])
test[factors] = scaler.fit_transform(test[factors])

### 建立模型

In [40]:
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve

In [41]:
def check_model(data, factors):
    
    classifier = lambda: SGDClassifier(
        loss='log',  # loss function: logistic regression
        penalty='elasticnet', # L1 & L2
        fit_intercept=True,  # 是否存在截距，默认存在
        # max_iter=100, 
        shuffle=True,  # Whether or not the training data should be shuffled after each epoch
        n_jobs=-1, # The number of processors to use
        class_weight=None) # Weights associated with classes. If not given, all classes are supposed to have weight one.
 
    # 管道机制使得参数集在新数据集（比如测试集）上的重复使用，管道机制实现了对全部步骤的流式化封装和管理。
    model = Pipeline(steps=[
        ('ss', StandardScaler()), # transformer
        ('en', classifier())  # estimator
    ])
 
    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }
 
    # StratifiedKFold用法类似Kfold，但是他是分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同。
    folder = StratifiedKFold(n_splits=5, shuffle=True)
    
    # Exhaustive search over specified parameter values for an estimator.
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1,  # -1 means using all processors
        verbose=1)
    
    grid_search = grid_search.fit(data[factors], 
                                  data['label'])
    
    return grid_search

### 训练

In [42]:
model = check_model(train, factors)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.0min finished


### 验证

对验证集中每个优惠券预测的结果计算 AUC，再对所有优惠券的 AUC 求平均。计算 AUC 的时候，如果 label 只有一类，就直接跳过，因为 AUC 无法计算。

In [43]:
y_pred = model.predict_proba(test[factors])

In [44]:
test_tmp = test.copy()
test_tmp['pred_prob'] = y_pred[:, 1]

In [45]:
test_tmp.sample(10)

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,Discount_type,Convert_rate,Discount_target,Discount_reduce,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label,pred_prob
743115,1410868,4663,11002,0.000000,20160528,null,1.0,0.812183,0.500000,0.20,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.031212
392585,6479918,450,1532,0.181818,20160523,null,1.0,0.761421,0.100000,0.05,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.079940
864608,4662241,760,2418,0.181818,20160603,null,1.0,0.761421,0.100000,0.05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.086461
1514879,2353001,1469,11848,0.272727,20160612,null,1.0,0.710660,0.333333,0.20,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.022161
1262278,996095,1798,6889,0.000000,20160525,20160603,1.0,0.634518,0.066667,0.05,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0.191463
149013,6020268,4043,3222,0.454545,20160518,null,1.0,0.761421,0.100000,0.05,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.056232
403656,2802747,5717,8192,0.181818,20160527,null,1.0,0.634518,0.066667,0.05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.108012
229150,1842797,1469,2601,0.090909,20160607,null,1.0,0.710660,0.333333,0.20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.047665
1594279,6526206,4663,13304,0.090909,20160530,null,1.0,0.812183,0.500000,0.20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.020496
1439267,6146853,6485,2079,0.090909,20160522,null,1.0,0.761421,0.100000,0.05,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.096618


#### 计算AUC

In [46]:
test_group = test_tmp.groupby(['Coupon_id'])
aucs = []
for i in test_group:
    tmp_df = i[1] 
    if len(tmp_df['label'].unique()) != 2:
        continue
    fpr, tpr, thresholds = roc_curve(tmp_df['label'], tmp_df['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))
print(np.average(aucs))

0.5331335604933403


## 测试

In [47]:
df_test[factors] = scaler.fit_transform(df_test[factors])

In [48]:
# test prediction for submission
y_test_pred = model.predict_proba(df_test[factors])
df_result = df_test[['User_id','Coupon_id','Date_received']].copy()
df_result['Probability'] = y_test_pred[:,1]
df_result.to_csv('submit1.csv', index=False, header=False)

In [49]:
df_result.sample(10)

,User_id,Coupon_id,Date_received,Probability
102215,1539872,9983,20160701,0.040583
105128,5500188,13602,20160705,0.172447
33398,865469,6390,20160727,0.137548
34143,5572173,12735,20160728,0.142959
81557,7144530,9983,20160728,0.117366
46574,3723975,12429,20160721,0.189363
6273,4915378,10418,20160728,0.077917
109041,2600690,13602,20160723,0.128958
25737,2856496,2978,20160706,0.202148
24621,4365352,8059,20160704,0.107598


## 保存模型 & 导入模型

In [50]:
import os
import pickle

In [51]:
if not os.path.isfile('1_model.pkl'):
    with open('1_model.pkl', 'wb') as f:
        pickle.dump(model, f)
else:
    with open('1_model.pkl', 'rb') as f:
        model = pickle.load(f)